In [ ]:
from __future__ import print_function

In [ ]:
# Short instruction to make sure we are working in the right environment
!conda info

In [23]:
#import libraries
import os, io
import tempfile
import time
import re
import pdf2image
from pdf2image import convert_from_path
from pdf2image import convert_from_bytes

from IPython.display import display, Image
import pytesseract
import tesserocr
import pdftotext

import pikepdf
from pikepdf import Pdf, Page
import pdfminer
import pdfplumber

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed

# to import from google drive
import pickle
import os.path
from googleapiclient import discovery
from googleapiclient.discovery import build
from googleapiclient import errors
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
from PIL import Image

In [7]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))


Files:
Copy of 11255.pdf (1HdCavlCXP-tAlaAmjPOm-i7efxUsiIsw)
Copy of Nº 004-2020-P-CE-PJ.pdf (1Y5jdAcakQfIP4A5H06c6lQoNBCFk9-me)
Copy of 3000.pdf (1rwaPoFock58hHlWIot3KEnUTuEl1p6n7)
Document de prova per pdf.pdf (1Z5-HSv2Ktox4UGLMM9MtJgJFCwZ5c70Z)
Pipelines (1Swlgw1QUnpdi9_PkR2JV8l5arsJ9rncQWx64yNASQEU)
Classification_Framework.docx (1oEhXfAjow-HPoVnqWp8mhfBfD0bwZ6H-)
Policy Accelerator - HSBC.pptx (11OMVjX0YfFF1GSN3iFmej0XFPQs3u12a)
Text Mining - Concept note for meetings.docx (1sFjkBNiccumsD09w5SAj5NlmChfkEAEg)
USAID.docx (1TZ2kL4T1xDXpWAgf1wYfXBU1qL47sbrR)
Microsoft Earth AI Draft JB.docx (1FLsFLuqnUh3nCfECotmQ56VZ8B3k45yr)


In [37]:
# 'application/vnd.google-apps.spreadsheet' and parents in '{}'".format(folder_id)
# "mimeType = 'application/vnd.google-apps.pdf' and parents in '{}'".format(folder_id)
folder_id = "1JU2YWwn88_0hwP5EYuF5FB2LLTYVZKND"
filetype = "application/pdf"
query = f"'{folder_id}' in parents"
query = f"mimeType='{filetype}'"
query = f"'{folder_id}' in parents and mimeType='{filetype}'"
page_token = None
while True:
    response = service.files().list(q = query,
                                    spaces="drive",
                                    fields="nextPageToken, files(id, name, mimeType)",
                                    pageToken=page_token).execute()
    for file in response.get('files', []):
        # Process change
        print('Found file: %s (%s)' % (file.get('name'), file.get('id')))
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break

Found file: Programa_comision_forestal_mex.pdf (10jCcgdxWFoO0AaQXwWk7RsA0RD77n4la)
Found file: Copy of 11255.pdf (1HdCavlCXP-tAlaAmjPOm-i7efxUsiIsw)
Found file: Copy of Nº 004-2020-P-CE-PJ.pdf (1Y5jdAcakQfIP4A5H06c6lQoNBCFk9-me)
Found file: Copy of 3000.pdf (1rwaPoFock58hHlWIot3KEnUTuEl1p6n7)
Found file: Document de prova per pdf.pdf (1Z5-HSv2Ktox4UGLMM9MtJgJFCwZ5c70Z)
Found file: Decreto_ejecutivo_57.pdf (1egVmcj6Ssiupf1mX3BEDBUkYBx7r1tfU)
Found file: Decreto_ejecutivo.pdf (1_qryhaX0h8obAyLsYzf-u_ph0WVTrL1X)
Found file: ORDENANZA REGULADORA DE TALA Y PODA DE NEJAPA.pdf (1QMPmYxavfGy2FiWc9yOuxRyfyEu66aj0)


In [38]:
file_id = ['10jCcgdxWFoO0AaQXwWk7RsA0RD77n4la', '1egVmcj6Ssiupf1mX3BEDBUkYBx7r1tfU', '1Z5-HSv2Ktox4UGLMM9MtJgJFCwZ5c70Z', '1HdCavlCXP-tAlaAmjPOm-i7efxUsiIsw', '1Y5jdAcakQfIP4A5H06c6lQoNBCFk9-me', '1rwaPoFock58hHlWIot3KEnUTuEl1p6n7']
filename = "Decreto_ejecutivo_57.pdf"

request = service.files().get_media(fileId=file_id[0])

fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [33]:
list_of_items = re.findall(r'<<(.*?)>>', fh.getvalue().decode('ISO-8859-1'))

In [34]:
len(list_of_items)
i = 0
for item in list_of_items:
    if "Image" in item or "image" in item:
        print(i, "--", item)
    i += 1
print("***", i, "elements")

11 -- /Type/XObject/ColorSpace/DeviceRGB/Subtype/Image/BitsPerComponent 8/Width 1274/Length 9587/Height 440/Filter/DCTDecode
29 -- /CropBox[0 0 488.976 779.528]/Parent 12 0 R/Contents 17 0 R/Type/Page/Resources<</ExtGState 35 0 R/ProcSet[/PDF/ImageC/Text]/Font 36 0 R
*** 34 elements


In [41]:
extracted_text = ""
parser = PDFParser(fh)
document = PDFDocument(parser)
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed
sio = io.StringIO()
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, sio, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)


for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
text = sio.getvalue()
    
#     layout = device.get_result()
#     for lt_obj in layout:
#         if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
#             extracted_text += lt_obj.get_text()
#         else:
#             print("no text to extract")

TypeError: unsupported operand type(s) for +=: '_io.StringIO' and 'int'

In [ ]:
file_id = '1egVmcj6Ssiupf1mX3BEDBUkYBx7r1tfU'
filename = "Decreto_ejecutivo_57.pdf"
def open_file_from_Gdrive(file_id):
    request = service.files().get_media(fileId=file_id)

    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))
    scan = convert_from_bytes(fh.getvalue())
    fh.close()
    
def check_file_content(bytesIO):
    bytesIO.getValue()

In [31]:
extracted_text

'Document de prova per pdf. \nA veure com funciona. Tres frases per provar. \n'

In [ ]:
fh.read()

In [ ]:
    gradient = np.linspace(0, 1, 256)
    gradient = np.vstack((gradient, gradient))
    image_data = cmap(gradient, bytes=True)
    image = Image.fromarray(image_data, 'RGBA')

    # ostream = io.FileIO('../cmaps/' + cmap_name + '.png', 'wb')
    # image.save(ostream, format='PNG')
    # ostream.close()

    ostream = io.BytesIO()
    image.save(ostream, format='PNG')
    cbar_png_bytes = ostream.getvalue()
    ostream.close()

    cbar_png_data = base64.b64encode(cbar_png_bytes)
    cbar_png_bytes = cbar_png_data.decode('unicode_escape')

    return cbar_png_bytes 

In [ ]:
# Mouting drive Google Drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
filename = "Decreto_ejecutivo_57.pdf" 
# inp_path = "/content/drive/MyDrive/WRI-LatinAmerica-Talent/Preprocessing/raw/"
# out_path = "/content/drive/MyDrive/WRI-LatinAmerica-Talent/Preprocessing/OCR/outputs"
# filepaths = [os.path.join(inp_path, file) for file in os.listdir(inp_path)]
# filepaths

# Checking whether the files are scanned images or true pdfs

In [ ]:
def is_image(file_path):
  with open(file_path, "rb") as f:
    return pdftotext.PDF(f)
    


In [ ]:
print(is_image(filename))

# Converting pdf to image files and improving quality

In [ ]:
def get_image1(file_path):
  """Get image out of pdf file_path. Splits pdf file into PIL images of each of its pages.
  """
  return convert_from_path(file_path, 500)

# Performance tips according to pdf2image: 
# Using an output folder is significantly faster if you are using an SSD. Otherwise i/o usually becomes the bottleneck.
# Using multiple threads can give you some gains but avoid more than 4 as this will cause i/o bottleneck (even on my NVMe SSD!).

In [ ]:
pages = get_image1(filepaths[0])
display(pages[0])

What can we do here to improve image quality? It already seems pretty good!

# Evaluating extraction time from each method and saving text to disk

In [ ]:
def export_ocr(text, file, extract, out=out_path):
  """ Export ocr output text using extract method to file at out
  """
  filename = f'{os.path.splitext(os.path.basename(file))[0]}_{extract}.txt'
  with open(os.path.join(out, filename), 'w') as the_file:
    the_file.write(text)

def wrap_pagenum(page_text, page_num):
  """ Wrap page_text with page_num tag
  """
  return f"<p n={page_num}>" + page_text + "</p>"

In [ ]:
# pytesseract extraction
start_time = time.time()
for file in filepaths:
  pages = get_image1(file)
  text = ""
  for pageNum, imgBlob in enumerate(pages):
    page_text = pytesseract.image_to_string(imgBlob, lang="spa")
    text += wrap_pagenum(page_text, pageNum)
  export_ocr(text, file, "pytesseract")  # write extracted text to disk
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# tesserocr extraction
start_time = time.time()
for file in filepaths:
  pages = get_image1(file)
  text = ""
  for pageNum, imgBlob in enumerate(pages):
    page_text = tesserocr.image_to_text(imgBlob, lang="spa")
    text += wrap_pagenum(page_text, pageNum)
  export_ocr(text, file, "tesserocr")  # write extracted text to disk
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# tesserocr extraction using the PyTessBaseAPI
start_time = time.time()
for file in filepaths:
  pages = get_image1(file)
  text = ""
  with tesserocr.PyTessBaseAPI(lang="spa") as api:
    for pageNum, imgBlob in enumerate(pages):
      api.SetImage(imgBlob)
      page_text = api.GetUTF8Text()
      text += wrap_pagenum(page_text, pageNum)
    export_ocr(text, file, "tesserocr_pytess")  # write extracted text to disk
print("--- %s seconds ---" % (time.time() - start_time))

It seems that the pytesseract package provides the fastest extraction and by looking at the extracted text it doesn't seem to exist any difference in the output of all the tested methods.

In [ ]:
# comparison between text extracted by the different methods
os.listdir(out_path)

In [ ]:
# TODO: perform a more programatical comparison between extracted texts

# Let's look at the extracted text

In [ ]:
with open(os.path.join(out_path, 'Decreto_ejecutivo_57_pytesseract.txt')) as text:
  extracted_text = text.read()
extracted_text

In [ ]:
# Replace \x0c (page break) by \n
# Match 1 or more occurrences of \n if preceeded by one occurrence of \n OR 
# Match 1 or more occurrences of \s (whitespace) if preceeded by one occurrence of \n OR 
# Match one occurrence of \n if it isn't followed by \n
print(re.sub("(?<=\n)\n+|(?<=\n)\s+|\n(?!\n)", " ", extracted_text.replace("\x0c", "\n")))